In [2]:
import cv2
import os
import numpy as np

# Create training data

In [3]:
def create_training_data(name, label, save_folder, num_samples):
    '''
    Creates the training data set by capturing a certain number of face samples from your video camera.
    The face detection model used for this part is the Haar Cascade model.
    '''
    face_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + "haarcascade_frontalface_default.xml" )

    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height
    
    count = 0

    while True:
        ret, img = cam.read()

        if ret == True:
            gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray_scale, 1.2, 4, minSize=(30, 30))

            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                count += 1

                # Save the image 
                cv2.imwrite(save_folder + "/" + label + "_" + name + "_" + str(count) + ".jpg", gray_scale[y:y+h, x:x+w])
                cv2.imshow('Create Training Data', img)
        
        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= int(num_samples): # Take num_samples face samples and stop video
            break

    cam.release()
    cv2.destroyAllWindows()
    print("Finished collecting samples.")


In [ ]:
name = input("Please enter the name of the person being recorded: ")
label = input("Please enter a label for this person: ")
num_samples = input("Please enter the number of samples you would like to collect: ")

create_training_data(name=name, label=label, save_folder="training_data", num_samples=num_samples)

# Load data

In [4]:
def load_data(data_folder):
    data = os.listdir(data_folder)

    names = []
    labels = []
    faces = []

    for img_name in data:
        name = img_name.split('_')[1]
        label = int(img_name.split('_')[0])

        image_path = data_folder + "/" + img_name

        img = cv2.imread(image_path)

        gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        names.append(name)
        labels.append(label)
        faces.append(cv2.resize(gray_scale, (254, 254)))

    return names, faces, labels

In [5]:
print("Loading training data...")
names, faces_raw, labels = load_data("training_data")
names = list(dict.fromkeys(names))
faces = np.array(faces_raw) / 255
print("Data prepared")

print(f"Number of names: {len(names)}")
print(f"Number of faces: {len(faces)}")
print(f"Number of labels: {len(labels)}")

number_of_classes = len(names)

Loading training data...
Data prepared
Number of names: 2
Number of faces: 600
Number of labels: 600


# LBPH Face Recognizer model

In [6]:
def train_lbph_recognizer(faces, labels):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(labels))

    return recognizer

# Train the face recognizer model and save the model
print("Training LBPH Face Recognizer model...")
lbph_model = train_lbph_recognizer(faces, labels)
print("Done training")

Training LBPH Face Recognizer model...
Done training


# LeNet-5 model

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam

lenet_model = Sequential()

# Layer 1
lenet_model.add(Conv2D(6, (5, 5), activation='relu', input_shape=(254, 254, 1)))
lenet_model.add(MaxPool2D((2, 2)))

# Layer 2
lenet_model.add(Conv2D(16, (5, 5), activation='relu'))
lenet_model.add(MaxPool2D((2, 2)))

# Flatten before dense layers
lenet_model.add(Flatten())

# Layer 3
lenet_model.add(Dense(120, activation='relu'))

# Layer 4
lenet_model.add(Dense(84, activation='relu'))

# Output layer
lenet_model.add(Dense(number_of_classes, activation='softmax'))

lenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

lenet_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 250, 250, 6)       156       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 125, 125, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 121, 121, 16)      2416      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 60, 60, 16)        0         
 ng2D)                                                           
                                                                 
 flatten_3 (Flatten)         (None, 57600)             0         
                                                                 
 dense_8 (Dense)             (None, 120)              

In [22]:
from keras.utils import to_categorical

y_data = to_categorical(np.array(labels), number_of_classes)
train_data = faces.reshape(np.array(faces).shape[0], 254, 254, 1)

lenet_model.fit(train_data, y_data, epochs=10, batch_size=32)

Epoch 1/10
19/19 [==============================] - 5s 254ms/step - loss: 0.6411 - accuracy: 0.7250
Epoch 2/10
19/19 [==============================] - 5s 280ms/step - loss: 0.1345 - accuracy: 0.9533
Epoch 3/10
19/19 [==============================] - 5s 276ms/step - loss: 0.0436 - accuracy: 0.9867
Epoch 4/10
19/19 [==============================] - 5s 243ms/step - loss: 0.0290 - accuracy: 0.9900
Epoch 5/10
19/19 [==============================] - 5s 240ms/step - loss: 0.0211 - accuracy: 0.9950
Epoch 6/10
19/19 [==============================] - 5s 244ms/step - loss: 0.0144 - accuracy: 0.9950
Epoch 7/10
19/19 [==============================] - 5s 277ms/step - loss: 0.0076 - accuracy: 0.9967
Epoch 8/10
19/19 [==============================] - 5s 264ms/step - loss: 0.0062 - accuracy: 0.9983
Epoch 9/10
19/19 [==============================] - 5s 247ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 10/10
19/19 [==============================] - 5s 252ms/step - loss: 0.0017 - accuracy: 1.0000

# AlexNet model

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout

alexnet_model = Sequential()

# Layer 1
alexnet_model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(254, 254, 1)))
alexnet_model.add(MaxPool2D((3, 3), strides=(2, 2)))

# Layer 2
alexnet_model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
alexnet_model.add(MaxPool2D((3, 3), strides=(2, 2)))

# Layer 3
alexnet_model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 4
alexnet_model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 5
alexnet_model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))

# Flatten before dense layers
alexnet_model.add(Flatten())

# Dense layers
alexnet_model.add(Dense(4096, activation='relu'))
alexnet_model.add(Dropout(0.5))
alexnet_model.add(Dense(4096, activation='relu'))
alexnet_model.add(Dropout(0.5))
alexnet_model.add(Dense(number_of_classes, activation='softmax'))

alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

alexnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 61, 61, 96)        11712     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 96)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 30, 30, 256)       614656    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 14, 14, 384)       885120    
                                                                 
 conv2d_16 (Conv2D)          (None, 14, 14, 384)      

In [18]:
from keras.utils import to_categorical

y_data = to_categorical(np.array(labels), number_of_classes)
train_data = faces.reshape(np.array(faces).shape[0], 254, 254, 1)

alexnet_model.fit(train_data, y_data, epochs=10, batch_size=64)

Epoch 1/10
10/10 [==============================] - 25s 3s/step - loss: 0.7030 - accuracy: 0.5083
Epoch 2/10
10/10 [==============================] - 25s 2s/step - loss: 0.3081 - accuracy: 0.8967
Epoch 3/10
10/10 [==============================] - 25s 2s/step - loss: 0.6603 - accuracy: 0.6600
Epoch 4/10
10/10 [==============================] - 25s 2s/step - loss: 0.3980 - accuracy: 0.8433
Epoch 5/10
10/10 [==============================] - 26s 3s/step - loss: 0.1548 - accuracy: 0.9383
Epoch 6/10
10/10 [==============================] - 26s 3s/step - loss: 0.0936 - accuracy: 0.9717
Epoch 7/10
10/10 [==============================] - 26s 3s/step - loss: 0.0379 - accuracy: 0.9900
Epoch 8/10
10/10 [==============================] - 26s 3s/step - loss: 0.0230 - accuracy: 0.9933
Epoch 9/10
10/10 [==============================] - 26s 3s/step - loss: 0.0104 - accuracy: 0.9983
Epoch 10/10
10/10 [==============================] - 26s 3s/step - loss: 0.0300 - accuracy: 0.9867


# Test the different models

In [10]:
def test_model(model, model_name, names):
    face_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + "haarcascade_frontalface_default.xml" )
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height

    while True:
        ret, img = cam.read()
        gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_scale, scaleFactor=1.2, minNeighbors=4, minSize=(30,30))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            face = gray_scale[y:y+h, x:x+w]
            face_resized = np.array(cv2.resize(face, (254, 254))) / 255

            if model_name == "lbph":
                id, confidence = model.predict(face_resized)

                if (confidence < 100):
                    name = names[id]
                    confidence = "  {0}%".format(round(100 - confidence))
                else:
                    name = "unknown"
                    confidence = "  {0}%".format(round(100 - confidence))
            else:
                confidence = ""
                face_resized = np.reshape(face_resized, (1, 254, 254, 1))
                id = int(np.argmax(model.predict(face_resized), axis=-1))
                name = names[id]

            cv2.putText(
                        img,
                        name,
                        (x+5, y-5),
                        font,
                        1,
                        (255, 255, 255),
                        2
                       )
            
            cv2.putText(
                        img,    
                        str(confidence), 
                        (x+5,y+h-5), 
                        font, 
                        1, 
                        (255,255,0), 
                        1
                       ) 
        
        cv2.imshow('Face Recognition', img)
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break

    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()


## LBPH model

In [17]:
test_model(model=lbph_model, model_name="lbph", names=names)


 [INFO] Exiting Program and cleanup stuff


## LeNet-5 model

In [23]:
test_model(model=lenet_model, model_name="lenet", names=names)

1/1 [==============================] - 0s 22ms/step

 [INFO] Exiting Program and cleanup stuff


## AlexNet model

In [24]:
test_model(model=alexnet_model, model_name="alexnet", names=names)

1/1 [==============================] - 0s 80ms/step

 [INFO] Exiting Program and cleanup stuff
